In [ ]:
#import pyod 
import pandas as pd
import numpy as np

In [ ]:
!python -m pip install pyod

In [ ]:
#wave_df=pd.read_csv("../data/2020-12-10-21-31-01-bedecg.preprocess.csv",names=['index','wave'])
#wave_df=pd.read_csv("../data/2020-12-10-21-31-01-bedecg.preprocess.csv") 
wave_df=pd.read_csv("1608219213000.csv")

In [ ]:
print(wave_df)

In [ ]:
!pip install statsmodels

In [ ]:
import statsmodels.api as sm
from statsmodels.nonparametric.kernel_regression import KernelReg

# FFT滤波去抖动噪声

In [ ]:
import scipy.fftpack
x=wave_df['Unnamed: 0']
y=wave_df['value']

N=1
w=scipy.fftpack.rfft(y)
f=scipy.fftpack.rfftfreq(N, x[1]-x[0])
spectrum = w**2

cutoff_idx = spectrum < (spectrum.max()/5)
w2 = w.copy()
w2[cutoff_idx] = 0

y2 = scipy.fftpack.irfft(w2)

In [ ]:
csize=1000
i=101
plt.plot(x[i*csize:(i+1)*csize],y2[i*csize:(i+1)*csize])
plt.show()

# 卷积去噪声

In [ ]:
x=wave_df['Unnamed: 0']
y=wave_df['value']

def smooth(y, box_pts):
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth

csize=50
i=20200
xplot=x[i*csize:(i+1)*csize]
yplot=y[i*csize:(i+1)*csize]
#plt.plot(xplot, yplot,'r-', lw=2)
plt.plot(xplot, smooth(yplot,3), 'r-', lw=2)
plt.plot(xplot, smooth(yplot,19), 'g-', lw=2)

smoothed=smooth(y,19)

# Lowess去噪声

In [ ]:
import statsmodels.api as sm

x=wave_df['Unnamed: 0']
y=wave_df['value']

lowess = sm.nonparametric.lowess(y, x, frac=0.1)

csize=50
i=20200
xplot=lowess[i*csize:(i+1)*csize, 0]
yplot=lowess[i*csize:(i+1)*csize, 1]
#plt.plot(xplot, yplot,'r-', lw=2)
plt.plot(xplot, yplot, 'r-', lw=2)
plt.show()

In [ ]:
x=wave_df['Unnamed: 0']
y=KernelReg(wave_df['value']
kr = KernelReg(y,x,'c')
y_pred, y_std = kr.fit(x)

In [ ]:
from scipy import signal

In [ ]:
peak_indx=signal.find_peaks(smoothed,distance=10)

In [ ]:
print(len(peak_indx[0]))
print(peak_indx[0][14:18])

In [ ]:
#peak_indx=[255]
size=500
wave_len=len(smoothed)
slide_peaks=np.zeros(wave_len,dtype=np.int)
#print(slide_peaks)
peak_used_indx=peak_indx[0]
#print(peak_used_indx)
for i in peak_used_indx:
    #print(i)
    i=int(i)
    li = i-size/2 if i-size/2>0 else 0
    ri = i+size/2 if i+size/2<wave_len-1 else wave_len-1
    #slide_peaks[li:ri]=slide_peaks[li:ri]+1
    j=int(li)
    #print(j)
    while j<=ri:
        #print(j)
        slide_peaks[j]=slide_peaks[j]+1
        j=j+1

print(slide_peaks)

In [ ]:
plt.plot(x, smoothed, 'g-', lw=2)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

#sns.lineplot(x = wave_df.index.values, y = 'value', palette= 'dark', data=wave_df) 
#plt.gcf().autofmt_xdate()
#plt.show()



In [ ]:
csize=1000
i=201
plt.plot(x[i*csize:(i+1)*csize],smoothed[i*csize:(i+1)*csize])
plt.show()

In [ ]:
from  pyod.models.copod  import COPOD
clf_name='COPOD'
clf=COPOD()
clf.fit(wave_df[6000000:16700000])

In [ ]:
# get the prediction labels and outlier scores of the training data
y_train_pred = clf.labels_  # binary labels (0: inliers, 1: outliers)
y_train_scores = clf.decision_scores_  # raw outlier scores

y_test_pred = clf.predict(wave_df)
y_test_scores = clf.decision_function(wave_df)

In [ ]:
print(wave_df)
test=pd.DataFrame()
test['value']=wave_df['value']
test['score']=y_test_scores*wave_df['value'].max()
test['result']=y_test_pred*wave_df['value'].max()
print(test)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.lineplot(x = test.index.values, y = 'value', palette= 'dark', data=test) 
plt.gcf().autofmt_xdate()
plt.show()

In [ ]:
size=test.index.size
csize=10000
count=size/csize+1
i=0

while i<=count:
    plt.plot(test.index.values[i*csize:(i+1)*csize],test['value'][i*csize:(i+1)*csize])
    plt.plot(test.index.values[i*csize:(i+1)*csize],test['score'][i*csize:(i+1)*csize])
    plt.show()
    i=i+1

In [ ]:
print(wave_df)
test1=pd.DataFrame()
test1['value']=wave_df['value']
test1['score']=y_test_scores
test1['score1']=test1['score'].rolling(500).mean()
test1['score2']=test1['value'].abs().rolling(500).mean()
#test1['score']=test1['score'].rolling(1000).mean()*wave_df['value'].max()/test1['score'].max()
#test1['result']=y_train_pred*wave_df['value'].max()
print(test1)

In [ ]:
size=test1.index.size
csize=5000
count=size/csize+1
i=0

while i<=count:
    plt.plot(test1.index.values[i*csize:(i+1)*csize],test1['value'][i*csize:(i+1)*csize])
    plt.show()
    plt.plot(test1.index.values[i*csize:(i+1)*csize],test1['score2'][i*csize:(i+1)*csize])
    plt.show()
    i=i+1

In [ ]:
plt.hist(test1['score2'])
plt.show()

In [ ]:
plt.plot(test.index.values[-500:],test['value'][-500:])
plt.plot(test.index.values[-500:],test['score'][-500:])
plt.show()

In [ ]:
plt.plot(y_train_scores)
plt.axhline(y=clf.threshold_, c='r', ls='dotted', label='threshoold')
plt.title('Anomaly Scores with automatically calculated threshold')

In [ ]:
!pip3 install keras

In [ ]:
from keras.utils.vis_utils import model_to_dot 
from IPython.display import SVG
SVG(model_to_dot(clf.model_, show_shapes=True, show_layer_names=True, rankdir='TB').create(prog='dot', format='svg'))


2. Single-Objective Generative Adversarial Active Learning.

In [ ]:
from  pyod.models.so_gaal  import SO_GAAL

In [ ]:
clf_name='so-gaal'
clf=SO_GAAL(stop_epochs=1)
clf.fit(wave_df)

In [ ]:
# get the prediction labels and outlier scores of the training data
y_train_pred = clf.labels_  # binary labels (0: inliers, 1: outliers)
y_train_scores = clf.decision_scores_  # raw outlier scores